In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt

In [11]:
pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.4/124.9 MB 13.4 MB/s eta 0:00:10
   ---------------------------------------- 0.9/124.9 MB 11.7 MB/s eta 0:00:11
   ---------------------------------------- 1.4/124.9 MB 11.4 MB/s eta 0:00:11
    --------------------------------------- 1.9/124.9 MB 11.1 MB/s eta 0:00:12
    --------------------------------------- 2.4/124.9 MB 11.0 MB/s eta 0:00:12
    --------------------------------------- 2.9/124.9 MB 10.9 MB/s eta 0:00:12
   - -------------------------------------- 3.4/124.9 MB 10.7 MB/s eta 0:00:12
   - -------------------------------------- 3.9/124.9 MB 10.9 MB/s eta 0:00:12
   - -------------------------------------- 4.4/124.9 MB 10.8 MB/s eta 0:00:12
   - -------------------------------------- 4.8/124.9 MB 11.0 MB/s eta 0:00:11
   - -------------------------------------- 5.3/124.9 MB 10.6 MB/s eta 0:00:12
   - -------------------------------------- 5.9/124.9 MB 10.

In [12]:
def selectkbest(indep_X,dep_Y,n):
        test = SelectKBest(score_func=chi2, k=n)
        fit1= test.fit(indep_X,dep_Y)
        # summarize scores       
        selectk_features = fit1.transform(indep_X)
        return selectk_features
    
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test
    
def r2_prediction(regressor,X_test,y_test):
     y_pred = regressor.predict(X_test)
     from sklearn.metrics import r2_score
     r2=r2_score(y_test,y_pred)
     return r2
 
def Linear(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LinearRegression
        regressor = LinearRegression()
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2   
    
def svm_linear(X_train,y_train,X_test):
                
        from sklearn.svm import SVR
        regressor = SVR(kernel = 'linear')
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
    
def svm_NL(X_train,y_train,X_test):
                
        from sklearn.svm import SVR
        regressor = SVR(kernel = 'rbf')
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
     

def Decision(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training setC
        from sklearn.tree import DecisionTreeRegressor
        regressor = DecisionTreeRegressor(random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
     

def random(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestRegressor
        regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2 

def xgboost(X_train,y_train,X_test):       
        from xgboost import XGBRegressor
        regressor = XGBRegressor(n_jobs=5,learning_rate=0.1,max_depth=10,random_state=1)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return r2

In [13]:
def rfeFeature(indep_X,dep_Y,n):
        rfelist=[]
        from sklearn.linear_model import LinearRegression
        lin = LinearRegression()
        from sklearn.svm import SVR
        SVRl = SVR(kernel = 'linear')
        from sklearn.svm import SVR
        #SVRnl = SVR(kernel = 'rbf')
        from sklearn.tree import DecisionTreeRegressor
        dec = DecisionTreeRegressor(random_state = 0)
        from sklearn.ensemble import RandomForestRegressor
        rf = RandomForestRegressor(n_estimators = 10, random_state = 0)
        rfemodellist=[lin,SVRl,dec,rf] 
        for i in   rfemodellist:
            print(i)
            log_rfe = RFE(i, n)
            log_fit = log_rfe.fit(indep_X, dep_Y)
            log_rfe_feature=log_fit.transform(indep_X)
            rfelist.append(log_rfe_feature)
        return rfelist

In [14]:
def rfe_regression(acclog,accsvml,accdes,accrf): 
    
    rfedataframe=pd.DataFrame(index=['Linear','SVC','Random','DecisionTree'],columns=['Linear','SVMl',
                                                                                        'Decision','Random'])

    for number,idex in enumerate(rfedataframe.index):

        rfedataframe['Linear'][idex]=acclog[number]       
        rfedataframe['SVMl'][idex]=accsvml[number]
        rfedataframe['Decision'][idex]=accdes[number]
        rfedataframe['Random'][idex]=accrf[number]
    return rfedataframe

In [15]:
dataset1=pd.read_csv("prep.csv",index_col=None)
df2=dataset1
df2 = pd.get_dummies(df2, dtype=int, drop_first=True)
indep_X=df2.drop('classification_yes', axis=1)
dep_Y=df2['classification_yes']

In [16]:
df2

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [17]:
def rfeFeature(indep_X, dep_Y, n):
    rfelist = []
    colnames_list = []  
    # List to store column names for each model
    r2_values = []  
    # List to store R2 values for each model

    from sklearn.linear_model import LinearRegression
    lin = LinearRegression()

    from sklearn.tree import DecisionTreeRegressor
    dec = DecisionTreeRegressor(random_state=0)

    from sklearn.ensemble import RandomForestRegressor
    rf = RandomForestRegressor(n_estimators=10, random_state=0)

    from xgboost import XGBRegressor
    xgb = XGBRegressor(n_jobs=5, learning_rate=0.1, max_depth=10, random_state=1)

    rfemodellist = [lin, dec, rf, xgb]

    for model in rfemodellist:
        log_rfe = RFE(estimator=model, n_features_to_select=n)
        log_fit = log_rfe.fit(indep_X, dep_Y)
        log_rfe_feature = log_fit.transform(indep_X)
        rfelist.append(log_rfe_feature)

        # Get the column names selected by RFE
        selected_columns = [col for col, selected in zip(indep_X.columns, log_rfe.support_) if selected]
        colnames_list.append(selected_columns)

        # Fit the model and calculate and store the R2 value
        X_train, X_test, y_train, y_test = split_scalar(pd.DataFrame(log_rfe_feature), dep_Y)
        model.fit(X_train, y_train)  # Fit the model
        r2 = r2_prediction(model, X_test, y_test)
        r2_values.append(r2)

    return rfelist, colnames_list, r2_values


In [18]:
# Call the function with your data
rfelist, colnames_list, r2_values = rfeFeature(indep_X, dep_Y, 3)

# Print the selected column names and R2 values for each model
for model_name, selected_columns, r2_value in zip(["Linear", "Decision", "Random", "XGBoost"], colnames_list, r2_values):
    print(f"Model: {model_name}")
    print("Selected Columns:", selected_columns)
    print(f"R2 Value: {r2_value}\n")

Model: Linear
Selected Columns: ['sg_c', 'sg_d', 'sg_e']
R2 Value: 0.44196072329151903

Model: Decision
Selected Columns: ['hrmo', 'sg_c', 'sg_d']
R2 Value: 0.9659612279458097

Model: Random
Selected Columns: ['al', 'hrmo', 'sg_d']
R2 Value: 0.8872560444374435

Model: XGBoost
Selected Columns: ['hrmo', 'sg_c', 'sg_d']
R2 Value: 0.9355477786097127



In [19]:
rfelist=rfeFeature(indep_X, dep_Y, 5)      
acclin=[]
accsvml=[]
accsvmnl=[]
accdes=[]
accrf=[]

In [20]:
rfelist

([array([[0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         ...,
         [0., 1., 0., 0., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0.]]),
  array([[ 57.48210526,  12.51815562,   4.70559701,   1.        ,
            0.        ],
         [ 22.        ,  10.7       ,   4.70559701,   1.        ,
            0.        ],
         [ 23.        ,  12.        ,   4.70559701,   0.        ,
            0.        ],
         ...,
         [115.        ,   9.1       ,   3.4       ,   1.        ,
            0.        ],
         [ 80.        ,   8.5       ,   4.70559701,   0.        ,
            0.        ],
         [ 49.        ,  16.3       ,   4.9       ,   0.        ,
            0.        ]]),
  array([[ 3.        , 12.51815562,  4.70559701,  1.        ,  0.        ],
         [ 2.        , 10.7       ,  4.70559701,  1.        ,  0.        ],
         [ 1.        , 12.        ,  4.70559701,  0.        ,  0.        ],
        

In [25]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep_X, dep_Y, test_size=0.30, random_state=0)
from sklearn.tree import DecisionTreeRegressor
regressor_dt=DecisionTreeRegressor(criterion='squared_error', splitter='random')
regressor_dt=regressor_dt.fit(x_train,y_train)

In [26]:
y_pred=regressor_dt.predict(x_test)

In [29]:
from sklearn.metrics import r2_score
r_score=r2_score(y_test,y_pred)

In [30]:
r_score

0.7866666666666666

In [31]:
from sklearn.ensemble import GradientBoostingRegressor
regressor_gbr = GradientBoostingRegressor(n_estimators=500,max_depth=4,min_samples_split=5,learning_rate=0.01,loss="squared_error")
regressor_gbr.fit(x_train, y_train)

GradientBoostingRegressor(learning_rate=0.01, max_depth=4, min_samples_split=5,
                          n_estimators=500)

In [32]:
y_pred=regressor_gbr.predict(x_test)

In [33]:
from sklearn.metrics import r2_score
r_score=r2_score(y_test,y_pred)
r_score

0.7718191118379065

In [34]:
from sklearn.ensemble import RandomForestRegressor
regressor_rf = RandomForestRegressor(n_estimators=100,random_state=0)
regressor_rf.fit(x_train, y_train)

RandomForestRegressor(random_state=0)

In [35]:
y_pred=regressor_rf.predict(x_test)

In [36]:
r_score

0.7718191118379065

In [37]:
# When comparing, Decision Tree algorithm gives maximum accuracy of 97%

In [38]:
import pickle
Finalised_Model="Finalized_model.sav"

In [39]:
pickle.dump(regressor_dt,open(Finalised_Model,'wb'))